# Getting Started with Ragas Experimental

> An introduction to using the experimental features of Ragas

In [ ]:
#| default_exp __init__

This notebook provides an introduction to the experimental features in Ragas.

In [ ]:
#| hide
from ragas_experimental import *